In [12]:
# violence post-processing
# violence class ID= 6
# person class ID = 1
# input data format : [class ID, track ID, x, y, width, height]
# output data format : [class ID, x, y, width, height]
# calculate the mid point of person bound box
# if the mid point is in the violence bound box, then the person is in violence
# if there are more than 2 persons in violence bound box, then it is violence

import sys
import os
import os.path

for _, _, files in os.walk('../samples/violence_sample_data/'):
    for file in files:
        with open('../samples/violence_sample_data/' + file, 'r') as f:
            input_file = f.read()

        input_data        = [x.split(' ') for x in input_file.split('\n')]     # data format : [class ID, track ID, x, y, width, height]
        res               = []                                                 # output data
        violences         = []                                                 # violence bound box
        person_mid_points = []                                                 # person mid point
        
        for _bound_box in input_data:
            class_id, track_id, x, y, width, height = _bound_box
            if _bound_box[0] == '6':
                violence_x_range = [float(_bound_box[1]), float(_bound_box[1]) + float(_bound_box[3])]
                violence_y_range = [float(_bound_box[2]), float(_bound_box[2]) + float(_bound_box[4])]
                violences.append([violence_x_range, violence_y_range])
                continue
            elif _bound_box[0] == '0':
                person_mid_points.append([float(_bound_box[1]) + float(_bound_box[3]) / 2, float(_bound_box[2]) + float(_bound_box[4]) / 2])
            res.append(_bound_box)

        for violence in violences:
            person_cnt = 0
            for mid_point in person_mid_points:
                if violence[0][0] < mid_point[0] < violence[0][1] and violence[1][0] < mid_point[1] < violence[1][1]:
                    person_cnt += 1
                if person_cnt >= 2:
                    res.append(['6', violence[0][0], violence[1][0], violence[0][1] - violence[0][0], violence[1][1] - violence[1][0]])
                    
        print(file,':',res)
        

violence_sample_positive.txt : [['0', '140.42', '57.25', '19.49', '37.93'], ['0', '162.94', '68.3', '22.09', '39.76'], ['6', 138.77, 55.31, 47.29999999999998, 53.78999999999999]]
violence_sample_negative.txt : [['0', '48.86', '66.2', '25.73', '36.66']]


In [1]:
# Dumping post-processing
# Person class ID = 0
# Trash class ID = 8
# Dumping class ID = 22
# calculate the distance between person and trash
# if the distance is less than 10, then it is a carried trash
# if the distance is greater than 10, then it is a dumping trash
# make a relation between person and trash

import sys
import os 
import os.path

from collections import defaultdict

def calculate_distance(x, y):
    return (((x[0] + x[2] / 2) - (y[0] + y[2] / 2)) ** 2 + ((x[1] + x[3] / 2) - (y[1] + y[3] / 2)) ** 2) ** 0.5

dumping_threshold  = 200

trashes            = {}                                                # trash track ID : [x, y, width, height]
frame_trashes      = defaultdict(lambda : [])
dumping_relations  = {}                                                # frame : {person track ID : [trash track ID]}
frame_relation     = defaultdict(lambda : [])                          # person track ID : [trash track ID]

for _, _, files in os.walk('../samples/dumping_sample_data/'):
    frame = 0
    for file in files:
        with open('../samples/dumping_sample_data/' + file, 'r') as f:
            input_file = f.read()
            
        input_data              = [x.split(' ') for x in input_file.split('\n')]    # data format : [class ID, track ID, x, y, width, height]
        res                     = []                                                # output data
        persons                 = {}                                                # person track ID : [x, y, width, height]
        
        trashes[frame]          = frame_trashes
        dumping_relations[frame]= frame_relation                                    # frame : {person track ID : [trash track ID]}
        
        for _bound_box in input_data:
            class_id, track_id, x, y, width, height = _bound_box
            if _bound_box[0] == '0':
                persons[track_id] = [float(x), float(y), float(width), float(height)]
            elif _bound_box[0] == '8':
                trashes[frame][track_id] = [float(x), float(y), float(width), float(height)]
            res.append(_bound_box)
            
            
        if frame != 0:
            for trash in trashes[frame]:
                if trash not in trashes[frame - 1]:
                    in_range_persons = list(filter(lambda x : calculate_distance(persons[x], trashes[frame][trash]) < dumping_threshold, persons))
                    for person_id in in_range_persons:
                        dumping_relations[frame][person_id].append(trash)
                else:
                    in_range_person = list(filter(lambda x : calculate_distance(persons[x], trashes[frame][trash]) < dumping_threshold, persons))
                    for person_id in in_range_person:
                        if trash not in dumping_relations[frame - 1][person_id]:
                            continue
                        dumping_relations[frame][person_id].append(trash)
                        
            for dumping_relation_person, dumping_relation_trash in dumping_relations[frame].items():
                if dumping_relation_trash == []:
                    continue
                if dumping_relation_person not in dumping_relations[frame - 1]:
                    continue
                else:
                    for trash in dumping_relation_trash:
                        if trash not in dumping_relations[frame - 1][dumping_relation_person]:
                            continue
                        res.append([file,'22', persons[dumping_relation_person][0], persons[dumping_relation_person][1], persons[dumping_relation_person][2], persons[dumping_relation_person][3]])
                        break
            
        frame += 1

print(res)

ValueError: not enough values to unpack (expected 6, got 5)